## Classical error correction

Information is an inherently physical object. To communicate it we must attach it to a physical manifestation such as speech, writing or actions. Any physical system can fail and when it does the information it carries can be lost or distorted.

Suppose Alice and Bob agree that Alice will turn the kitchen light on if she is working from home and turn it off otherwise. The idea is efficient but it depends entirely on the light functioning. If the bulb fails, Bob may draw the wrong conclusion despite Alice’s intention remaining unchanged. To reduce this risk Alice could instead turn on the kitchen, bedroom and bathroom lights. It is unlikely that all three bulbs fail at once, so Bob is far more likely to receive the intended message.

This is an example of classical error correction. Using multiple bulbs introduces redundancy into the communication channel so that even when parts of the physical system fail the original message can still be recovered. The goal of error correction is to introduce redundancy and structure into the communication channel so that the information remains intact despite physical imperfections.

### Information as binary strings

We can encode the light switch example using binary numbers. The switch is either on or off, which we encode as 
$1$ for on and $0$ for off. This is an example of a bit. A bit represents a physical system with two states, labelled $0$ and $1$.

Errors can occur when a bit changes state: for example a $0$ becomes a $1$ or a $1$ becomes a $0$. These are called bit-flip errors. For example, if Alice tries to send $1$ by turning the light on but the bulb fails, Bob will interpret it as a $0$ and misread the message.

A simple way to reduce errors is to increase redundancy by sending multiple copies of the same bit. We then assume that the value appearing most often is the intended message. For example, suppose that Alice sends $111$ and each bulb has probability $0.1$ of failure. Suppose that Bob receives $101$. The probability that only the middle bit flipped is $0.9 \cdot 0.1 \cdot 0.9 = 0.081$, whereas the probability the other two bits flipped is $0.1 \cdot 0.9 \cdot 0.1 = 0.009$. Therefore, it is much more likely that only one bit flipped and the intended message was $111$. In general, the most likely codeword is the one where the fewest bits have flipped, which corresponds to the majority of bits. This method is called majority decoding.

The more copies Alice sends the more likely Bob will choose the intended message. However, there is a trade-off. Sending more copies also increases the number of bits Bob must check, which takes more time and reduces efficiency. We can see this in the following example.

#### Example. 

First let us create a noisy channel over which to send bits.

In [ ]:
import random

error_prob = 0.25

def send_bit(bit, error_prob) -> int:
    """ 
    Sends one bit across a noisy channel.
    Each bit has probability error_prob at being flipped.
    """
    if random.random() < error_prob:
        return (bit + 1) % 2
    return bit

send_bit_trials = 10
error_prob = 0.2
for send_bit_trial in range(send_bit_trials):
    bit = random.choice([0,1])
    received_bit = send_bit(bit, error_prob)
    print(f"Sent bit: {bit}. Received bit: {received_bit}. Error occured? {bit != received_bit}.")

To improve reliability, we can increase the number of light bulbs, or equivalently increase the number of bits in our binary strings. Let us consider the case with three light switches.

In [ ]:
def decode_message(bits):
    return 1 if sum(bits) > len(bits)//2 else 0

print(f"\nThree light switches")

send_three_bit_trials = 5
error_prob = 0.2
num_correct = 0
for _ in range(send_three_bit_trials): 
    alice_intended = random.choice([0,1])
    alice_message = [alice_intended]*3
    bob_message = [send_bit(bit, error_prob) for bit in alice_message]
    bob_interpretation = decode_message(bob_message)
    print(f"\nAlice intended: {alice_intended}. Alice sent: {alice_message}. Bob received: {bob_message}. Bob interpreted: {bob_interpretation}.")
    print(f"    Interpretation correct? {alice_intended == bob_interpretation}.")
    if alice_intended == bob_interpretation:
        num_correct += 1
print(f"\nPercentage of correct interpretations: {float(num_correct/send_three_bit_trials)}.")



However, as the number of bits increases, the time required to decode the message also grows, since there are more bits to check.

In [ ]:
import matplotlib.pyplot as plt
import time

error_prob = 0.2 
num_trials = 500000
num_bits = range(1, 31, 2)
success_probs = []
decode_times = []

for n in num_bits:
    successes = 0
    total_time = 0

    for trial in range(num_trials):
        start_time = time.time()
        alice_intention = random.choice([0,1])
        alice_message = [alice_intention]*n
        bob_message = [send_bit(i, error_prob) for i in alice_message]
        bob_interpretation = decode_message(bob_message)
        trial_time = time.time() - start_time
        total_time += trial_time
        if bob_interpretation == alice_intention:
            successes += 1

    success_prob = float(successes/num_trials)
    success_probs.append(success_prob)

    decode_time = float(total_time/num_trials)
    decode_times.append(decode_time)

fig, (ax1, ax2) = plt.subplots(2, sharex=True)
fig.suptitle("Majority vote decoding vs number of bits", fontsize=12)

ax1.plot(num_bits, success_probs)
ax1.set(
    ylabel="Probability of correct decoding",
    #title=f"Probability correct message is decoded using majority vote vs number of encoding bits. \n Error prob = {error_prob}."
    )
ax1.grid(visible=True)

ax2.plot(num_bits, decode_times)
ax2.set(
    xlabel="Number of bits used to encode message",
    ylabel="Time taken to decode",
    #title=f"Time taken to decode message using majority vote vs number of encoding bits. \n Error prob = {error_prob}."
    )
ax2.grid(visible=True)

plt.show

Ideally we want Bob to receive the message Alice intended with high probability as efficiently as possible.

### Encoding messages

Sometimes we want to communicate multiple pieces of information at once. Suppose Bob needs to know whether Alice is working from home today and tomorrow. Alice could use two switches, one for each day, sending the binary string $10$ if she is working today but not tomorrow.

Again to reduce the risk of errors, Alice can add redundancy. For example, she could send six bits instead of two, encoding $10$ as $111000$. Now Alice can send one of four possible binary strings: $$ 000000, 000111, 111000, 111111.$$ We can generalise this idea. Let $\mathcal{M}$ be the set of messages Alice wants to send. Choose an integer $n \in \{0,1\}^n$ and define an injective function $$ E \colon \mathcal{M} \to \{0,1\}^n. $$ Note you must choose $n$ so that $2^n$ is larger than the number of messages in $\mathcal{M}$ for such an injective function to exist. This function is shared with Bob to allow him to infer the intended message from the received string.

Of course, we still have the same issue: Bob may not receive one of the four binary strings sent by Alice. In fact, Bob could receive any $n$-bit string. Suppose Alice sends one of the four 6-bit codewords above and Bob receives $010111$. Previously we counted the most frequent value in the binary string to obtain the most likely codewood. However, now if we compare $010111$ to the possible messages sent by Alice:
| Sent binary string | Number of bit flips to get to $010111$ |
| ---                | ---                                    |
| $000000$ | 4 |
| $000111$ | 1 |
| $111000$ | 5 |
| $111111$ | 2 |

Therefore, we assume the most likely binary string that Alice sent is $000111$ and Bob takes this to be the intended message. 

This example can be generalised to obtain _error correcting codes_ by interpreting $\{0,1\}^n$ as a vector space and using linear algebra.

### Error correction codes

Fix an integer $n \in \mathbb{Z}_{>0}$. The binary strings of length $n$ are precisely the vectors in the vector space $(\mathbb{Z}/2\mathbb{Z})^n$. A _(binary) linear code_ of _length_ $n$ is a vector subspace $$\mathcal{C} \subseteq (\mathbb{Z}/2\mathbb{Z})^n.$$  An element of the code $\mathcal{C}$ is called a _codeword_. The _rank_ of a code is its dimension as a vector space over $\mathbb{Z}/2\mathbb{Z}$.

#### Examples.

1. The set $\mathcal{C}_1 = \{(0,0,0), (1,1,1)\}$ is a vector subspace of $(\mathbb{Z}/2\mathbb{Z})^3$, and hence a code. The basis of the code is $\{(1,1,1)\}$. Thus, $\mathcal{C}_1$ is a code of length $3$ and rank $1$ that contains $2$ codewords.

2. The set $$\mathcal{C}_2 = \{(0,0,0),(1,1,0),(1,0,1),(0,1,1)\}$$ also forms a subspace of $(\mathbb{Z}/2\mathbb{Z})^3$, and so is a code. One basis of $\mathcal{C}_2$ is $\{(1,1,0),(1,0,1)\}$. Thus, $\mathcal{C}_2$ is a code of length $3$ and rank $2$ that contains $4$ codewords.

3. For any subset $S$ of $(\mathbb{Z}/2\mathbb{Z})^n$ the span $$\mathcal{C}(S) = \mathrm{Span}_{\mathbb{Z}/2\mathbb{Z}}(S)$$ defines a binary code of length $n$. The rank and number of codewords of $\mathcal{C}(S)$ depends on the linear relations between the elements of $S$.

Since a code is a vector subspace, a code is uniquely determined by a basis. A _generator matrix_ of a code is a matrix whose rows form a basis of the code. Note that a basis of a vector space is not unique and hence the generator matrix of a code is not unique either.

#### Examples.

1. One generator matrix for $\mathcal{C}_1$ is
$$ \begin{pmatrix} 1 & 1 & 1 \end{pmatrix}. $$

2. One generator matrix for $\mathcal{C}_2$ is
$$ \begin{pmatrix} 1 & 1 & 0 \\ 1 & 0 & 1 \end{pmatrix}. $$

In [ ]:
import random

from linearcodes import LinearCode

# C1 = {000,111}

M1 = LinearCode([[0,0,0],[1,1,1]])
print(f"\nC1 has codewords: {M1.elements};")
print(f"  Length: {M1.length};")
print(f"  Rank: {M1.rank};")
G1 = M1.generator_matrix
print(f"  Generator matrix: \n{G1.array}.")

# C2 = {000,110,101,011}

M2 = LinearCode([[0,0,0],[1,1,0],[1,0,1],[0,1,1]])
print(f"\nC2 has codewords: {M2.elements};")
print(f"  Length: {M2.length};")
print(f"  Rank: {M2.rank};")
G2 = M2.generator_matrix
print(f"  Generator matrix: \n{G2.array}.")

# A random code C3

num_gen_elts = 3
binary_dimension = 5
elts = []
for elt in range(num_gen_elts):
    elt = []
    for bit in range(binary_dimension):
        elt.append(random.choice([0,1]))
    elts.append(elt)

M3 = LinearCode(elts)
codewords = M3.elements
print(f"\nC3 has {len(codewords)} codewords:")
print(f"  {codewords};")
print(f"  Length: {M3.length};")
print(f"  Rank: {M3.rank};")
G3 = M3.generator_matrix
print(f"  Generator matrix: \n{G3.array}.")


The generator matrix of a code $\mathcal{C}$ of length $n \in \mathbb{Z}_{>0}$ defines a linear map
$$G \colon (\mathbb{Z}/2\mathbb{Z})^n \to \mathcal{C}.$$
The kernel of this map is a vector subspace of $(\mathbb{Z}/2\mathbb{Z})^n$ and therefore forms another code. We call $\ker(G)$ the dual code of $\mathcal{C}$ and denote it by $\mathcal{C}^{\perp}$.

There is an equivalent characterisation of the dual code:
$$\mathcal{C}^{\perp} = \{ v \in (\mathbb{Z}/2\mathbb{Z})^n \mid c \cdot v = 0 \text{ for all } c \in \mathcal{C} \}.$$
These definitions agree because a vector $v$ satisfies $Gv = 0$ exactly when it is orthogonal to every row of the generator matrix, and these rows form a basis for $\mathcal{C}$. By the Rank Nullity Theorem, if $\mathcal{C}$ has length $n$ and rank $k$, then $\mathcal{C}^{\perp}$ has rank $n-k$. A generator matrix for $\ker(G)$ is called a parity check matrix of $\mathcal{C}$, denoted by $H$.

In summary we have:
| Code $\mathcal{C}$ | Dual code $\mathcal{C}^\perp$ |
| ---               | ---                           |
| Generator matrix $G$ | Generator matrix $H$ |
| Parity check matrix $H$ | Parity check matrix $G$ |

#### Examples.

1. The code $\mathcal{C}_1$ with generator matrix $$ G_1 = \begin{pmatrix} 1 & 1 & 1 \end{pmatrix}$$ has a parity check matrix $$ H_1 = \begin{pmatrix} 1 & 1 & 0 \\ 1 & 0 & 1 \end{pmatrix}. $$

2. The code $\mathcal{C}_2$ with generator matrix $$ G_2 = \begin{pmatrix} 1 & 1 & 0 \\ 1 & 0 & 1 \end{pmatrix}$$ has a parity check matrix  $$ H_2 = \begin{pmatrix} 1 & 1 & 1 \end{pmatrix}$$

In particular, the dual code of $\mathcal{C}_1$ is $\mathcal{C}_2$ and vice versa.






The parity check matrix detects whether a vector belongs to the code. A vector $v$ lies in $\mathcal{C}$ exactly when
$$Hv^{\mathrm{T}} = 0.$$
Each row of $H$ imposes a parity condition on a subset of the coordinates of $v$. The sum of the selected coordinates must be even, which corresponds to the parity check returning $0$.

The parity check matrix also allows us to detect errors in transmission. Errors are bit flips: a $0$ becomes a $1$ or a $1$ becomes a $0$. In $(\mathbb{Z}/2\mathbb{Z})^n$ this is the same as adding an error vector $e$ with a $1$ in each position where a bit has flipped. If Alice intends to send a codeword $c$ but Bob receives
$$m = c + e,$$
then applying the parity check matrix gives
$Hm^{\mathrm{T}} = Hc^{\mathrm{T}} + He^{\mathrm{T}} = He^{\mathrm{T}}.$
Since $Hc^{\mathrm{T}} = 0$, the vector $Hm^{\mathrm{T}}$ depends only on the error pattern. This vector is called the syndrome and it forms the basis of syndrome decoding.


##### Tanner Graph

The _Tanner graph_ of the code is a bipartite graph whose vertex sets are:
1. The set of bits in a codeword.
2. The parity check equations for the code.
Two vertices are connected if that bit is included in the parity check equation.

### Error correction



Linear codes are a classical technique for error correction in message transmission. They work by adding redundancy to messages so that errors can be detected and corrected. We have already seen an example of this with the code 
$$
\mathcal{C}_1 = \{000, 111\}.
$$

##### Encoding

Recall that Alice wants to tell Bob whether she is working. Alice could send Bob a single bit of information, either $0$ or $1$, but in this case, any error during transmission could cause Bob to misinterpret the message. To reduce this risk, Alice instead sends three identical bits: $000$ for $0$ and $111$ for $1$. Formally, she encodes the message space 
$$
\mathcal{M} = \{0,1\}
$$ 
as codewords of $\mathcal{C}_1$ using a bijective map
$$
E \colon \mathcal{M} \to \mathcal{C}_1
$$
where $E(0) = 000$ and $E(1) = 111$, which is shared with Bob.



### Maximum likelihood decoder

When a codeword is sent through a physical channel, some bits may be corrupted. As a result, the received message might differ from the original codeword. Decoding aims to recover the original codeword despite these errors. Without knowing the exact errors that occurred, we cannot be certain that decoding will find the intended message. So instead we try to find the codeword that is most likely to have been sent. More precisely, let $\mathcal{C}$ be a code of length $n \in \mathbb{Z}_{>0}$ and suppose we receive a message $m \in \{0,1\}^n$. Then we wish to find a codeword $\hat{c} \in \mathcal{C}$ such that 
\begin{equation*}
    \mathbb{P}(\text{ Sent } \hat{c} \mid \text{ Received } m) = \mathrm{max}_{c \in \mathcal{C}} \{\mathbb{P}(\text{ Sent } c \mid \text{ Received } m)\}.
\end{equation*}

We denote $\mathbb{P}(\text{ Sent } c \mid \text{ Received } m)$ by  $\mathbb{P}(c \mid m)$. 
The application of Bayes Theorem to $\mathbb{P}(c \mid m)$ results in
$$\mathbb{P}(c \mid m) = \frac{\mathbb{P}(m \mid c)\cdot \mathbb{P}(c)}{\mathbb{P}(m)}.$$
Notice that $\mathbb{P}(m)$ does not depend on the specific codeword $c$, and so, to maximise $\mathbb{P}(c \mid m)$ over the code $\mathcal{C}$ it suffices to maximise $$\mathbb{P}(m \mid c)\cdot \mathbb{P}(c)$$ over $\mathcal{C}$. To do this, we first make a couple of assumptions:

1. **Uniform codeword distribution:** Each codeword is equally likely to be sent. That is, for all codewords $c \in \mathcal{C}$ we have that $$\mathbb{P}(c) = \frac{1}{|\mathcal{C}|}.$$ Thus, maximising $\mathbb{P}(m \mid c)\cdot \mathbb{P}(c)$ over $\mathcal{C}$ is equivalent to maximising $\mathbb{P}(m \mid c)$ over $\mathcal{C}$.

2. **All errors are bit-flips:** A codeword $c \in \mathcal{C}$ produces a message $m$ that is a binary string of the same length. 

3. **Bit-flips are independent:** Each bit flips independently of others. That is, for a message $m=m_1m_2\dots m_n$ and a codeword $c=c_1c_2\dots c_n$, we have that
    $$\mathbb{P}(m \mid c) = \prod_{i=1}^n \mathbb{P}(m_i \mid c_i).$$
    In particular, since both $m_i$ and $c_i$ are bits, we have that:
    $$ \mathbb{P}(m_i \mid c_i) = \begin{cases} \mathbb{P}(c_i \text{ not flipped}) &\text{ if } m_i=c_i \\ \mathbb{P}(c_i \text{ flipped to } m_i) &\text{ if } m_i \neq c_i. \end{cases} $$

Therefore, to compute $\mathbb{P}(c \mid m)$ for any codeword $c$ and message $m$ of length $n \in \mathbb{Z}_{>0}$, it suffices to know for each $1 \leq i \leq n$, the probabilities
$$ \mathbb{P}(c_i \text{ flips } 0 \to 1) \; \text{ and } \; \mathbb{P}(c_i \text{ flips } 1 \to 0). $$
A _binary symmetric channel_ is a channel where these two probabilities coincide and a bit flips with the same probability from $0$ to $1$ as it does from $1$ to $0$.

---
##### Example.

Let us consider the code $\mathcal{C} = \{000, 111\}$. Suppose that we use a binary symmetric channel and each bit has a probability $p=\frac{1}{4}$ of flipping regardless of position. Suppose that we receive the message $m=100$ and we wish to compute the most likely codeword it came from. Since maximising $\mathbb{P}(c \mid m)$ over $\mathcal{C}$ is equivalent to maximising $\mathbb{P}(m \mid c)$ over $\mathcal{C}$, it suffices to compute $\mathbb{P}(m \mid c)$ for each codeword $c \in \mathcal{C}$. That is, for $c=000$ and $c=111$.

First suppose the codeword $c$ was $000$. Then receiving the message $m=100$ means the first bit flipped and the other two did not flip (TODO: can a bit flip more than once?). In particular, we have
\begin{align*}
    \mathbb{P}(m=100 \mid c=000) &= \mathbb{P}(m_1=1 \mid c_1=0) \cdot \mathbb{P}(m_2=0 \mid c_2=0) \cdot \mathbb{P}(m_3=0 \mid c_3=0) \\ &= \mathbb{P}(c_1 \text{ did flip}) \cdot \mathbb{P}(c_2 \text{ did not flip}) \cdot \mathbb{P}(c_3 \text{ did not flip}) \\ &= \frac{1}{4} \cdot \frac{3}{4} \cdot \frac{3}{4} = \frac{9}{64}. 
\end{align*}
Now suppose the codeword $c$ was $111$, then receiving the message $m=100$ means the first bit did not flip and the other two did flip, so we have
\begin{equation*}
    \mathbb{P}(m=100 \mid c=111) = \frac{3}{4} \cdot \frac{1}{4} \cdot \frac{1}{4} = \frac{3}{64}. 
\end{equation*}
Therefore, as $\mathbb{P}(m=100 \mid c=000)$ is larger than $\mathbb{P}(m=100 \mid c=111)$ it is most likely that the message $m=100$ came from the codeword $c=000$.
<!-- 
Therefore, we have that
\begin{align*}
    \sum_{b \in \mathcal{C}} \mathbb{P}(m \mid b) &= P(m \mid c=000) + P(m \mid c=111) 
    \\
    &= \frac{9}{64} + \frac{3}{64} = \frac{3}{16}.
\end{align*}
Thus, substituting these probabilities in, it follows that
$$\mathbb{P}(c=000 \mid m) = \frac{1}{|\mathcal{C}|} \cdot \frac{\mathbb{P}(m \mid c=000)}{\sum_{d \in \mathcal{C}} \mathbb{P}(m \mid d)} =  \frac{1}{1}\cdot \frac{\frac{9}{64}}{\frac{3}{16}} = \frac{3}{4},$$
and
$$\mathbb{P}(c=111 \mid m) = \frac{1}{|\mathcal{C}|} \cdot \frac{\mathbb{P}(m \mid c=111)}{\sum_{d \in \mathcal{C}} \mathbb{P}(m \mid d)} =  \frac{1}{1}\cdot \frac{\frac{3}{64}}{\frac{3}{16}} = \frac{1}{4}.$$
Therefore, if $m=100$ is the transmitted message, then the most likely codeword that was sent is $c=000$ with a probability of $\frac{3}{4}$. -->

We can compute the most likely codeword for each of the possible messages $m \in (\mathbb{Z}/2\mathbb{Z})^3$.

<!-- | Message $m$   | $\mathbb{P}(m \mid c=000)$ | $\mathbb{P}(m \mid c=111)$ | $\sum_{b \in \mathcal{C}} \mathbb{P}(m \mid b)$ | $\mathbb{P}(c=000 \mid m)$ | $\mathbb{P}(c=111 \mid m)$ |
| ---                    | ---                    | ---                    | ---             |---                    | ---                    |
| 000                    | $\frac{27}{64}$        | $\frac{1}{64}$         | $\frac{7}{16}$  | $\frac{27}{28}$        | $\frac{1}{28}$         |
| 100, 010, 001          | $\frac{9}{64}$         | $\frac{3}{64}$         | $\frac{3}{16}$  | $\frac{3}{4}$         | $\frac{1}{4}$         |
| 110, 101, 011          | $\frac{3}{64}$         | $\frac{9}{64}$         | $\frac{3}{16}$  | $\frac{1}{4}$         | $\frac{3}{4}$         |
| 111                    | $\frac{1}{64}$         | $\frac{27}{64}$        | $\frac{7}{16}$  | $\frac{1}{28}$         | $\frac{27}{28}$        |  -->

| Message $m$   | $\mathbb{P}(m \mid c=000)$ | $\mathbb{P}(m \mid c=111)$  | Most likely codeword $c$ |
| ---                    | ---                    | ---                    | ---             |
| 000                    | $\frac{27}{64}$        | $\frac{1}{64}$         | 000 <!-- with probability $\frac{27}{28}$ --> |
| 100, 010, 001          | $\frac{9}{64}$         | $\frac{3}{64}$         | 000 <!-- with probability $\frac{3}{4}$ --> |
| 110, 101, 011          | $\frac{3}{64}$         | $\frac{9}{64}$         | 111 <!-- with probability $\frac{3}{4}$ --> |
| 111                    | $\frac{1}{64}$         | $\frac{27}{64}$        | 111 <!-- with probability $\frac{27}{28}$ --> |


---


In [ ]:
from linearcodes import Codeword, LinearCode, HammingCode
import random
import numpy as np
import time

def send_codeword(
        code_length: int,
        channel_probabilities: list[float],
        codeword: Codeword):
    if len(channel_probabilities) != code_length:
        raise ValueError(f"List of probabilities must be of length code_length={code_length}. Got {len(channel_probabilities)}.")
    if len(codeword) != code_length:
        raise ValueError(f"Codeword must of length code_length={code_length}. Got {len(codeword)}.")
    message = []
    for bit_idx in range(code_length):
        p = channel_probabilities[bit_idx]
        c = codeword.vector[bit_idx]
        r = np.random.uniform(0,1)
        if r < p:
            flip_bit = (c+1)%2
            message.append(flip_bit)
        else:
            message.append(c)
    return message

def maximum_likelihood_decoder(channel_probabilities: list[float],
                               code: LinearCode, 
                               message: list[int]) -> Codeword:
    """
    Returns the codeword that was most likely to be sent given message is received.
    Args:
        - channel_probabilities (list[float]):
            entry with index i is the probability bit i will flip.
        - code (lc.LinearCode):
            code used for encoding.
        - message (list[int]):
            received message.
    """
    n = code.length
    if len(channel_probabilities) != n:
        raise ValueError(f"List of probabilities must be of length {n}. Got {len(channel_probabilities)}.")
    if len(message) != n:
        raise ValueError(f"Message must be of length {n}. Got {len(message)}.")
    codewords = code.codewords
    most_likely_codeword = Codeword([0]*n)
    most_likely_codeword_prob = 0
    for c in codewords:       
        c_probability = 1
        for bit_idx in range(n):
            p = channel_probabilities[bit_idx]
            if c.vector[bit_idx] != message[bit_idx]:
                c_probability *= p
            else:
                c_probability *= (1-p)
        if c_probability > most_likely_codeword_prob:
            most_likely_codeword = c
            most_likely_codeword_prob = c_probability
    return most_likely_codeword

code = HammingCode(3)
code_length = code.length
codewords = code.codewords
channel_probabilities = [float(0.1)]*code_length
results: dict[int, tuple[bool, float]] = {}

num_trials = 10
for trial in range(num_trials):
    print(f"\n=== Maximimum likelihood decoder {trial+1} ===")
    codeword = random.choice(codewords)

    message = send_codeword(code_length, channel_probabilities, codeword)
    start_time = time.time()
    most_likely_codeword = maximum_likelihood_decoder(channel_probabilities, code, message)
    decode_time = time.time() - start_time

    # print(f"\nRows are: sent codeword c, received message m, most likely sent codeword c'.")
    data = np.array([codeword.vector, message, most_likely_codeword.vector])
    # print(data)

    decode_correct = codeword == most_likely_codeword
    # print(f"\nDecoding correct? {decode_correct}")
    # print(f"Actual number of errors: {sum(1 for bit_idx in range(code_length) if codeword.vector[bit_idx] != message[bit_idx])}")
    # print(f"Guessed number of errors: {sum(1 for bit_idx in range(code_length) if most_likely_codeword.vector[bit_idx] != message[bit_idx])}")

    results[trial] = (decode_correct, decode_time)
    print(f"Decode correct? {decode_correct}")
    print(f"Time to decode = {decode_time*(10**5):.3f}x10^5")


#### Hamming distance

We can simplify the Maximum Likelihood Decoder by using the Hamming distance. Recall maximising $\mathbb{P}(c \mid m)$ is equivalent to maximising $\mathbb{P}(m \mid c) = \prod_{i=1}^n \mathbb{P}(m_i \mid c_i)$. Let us assume we have a binary symmetric channel such that the probability the probability a bit flips is $p$. Therefore, we have that
$$
\mathbb{P}(m_i \mid c_i) =
\begin{cases}
p & \text{if } m_i \neq c_i, \\
1-p & \text{if } m_i = c_i.
\end{cases}
$$
Notice that the probabilities depend only whether the bits are the same or different. In particular, the power of $p$ in the product $\prod_{i=1}^n \mathbb{P}(m_i \mid c_i)$ is precisely the number of bits in $m$ that differ from the bits in $c$, that is the Hamming distance $d(m,c)$. Therefore, we have that 
$$\mathbb{P}(m \mid c) = \prod_{i=1}^n \mathbb{P}(m_i \mid c_i) = p^{d(m,c)}(1-p)^{n-d(m,c)}.$$
Rearranging this expression we obtain
$$\mathbb{P}(m \mid c) = \left( \frac{p}{1-p}\right)^{d(m,c)}  \cdot (1-p)^{n}.$$
Note that $(1-p)^{n}$ depends neither on the codeword $c$ nor the message $m$. Thus, maximising $\mathbb{P}(m \mid c)$ over $\mathcal{C}$ is equivalent to maximising
$$\left( \frac{p}{1-p}\right)^{d(m,c)}$$
over $\mathcal{C}$.

By convention we assume that $p < \frac{1}{2}$. If it is not, then we can assume the received message is more likely to be wrong and flip all the bits before working with them .... (Fix this phrasing!). Consequently, the value $\frac{p}{1-p}$ is less than $1$ and so to maximise $$\left( \frac{p}{1-p}\right)^{d(m,c)}$$ we must minimise $d(m,c)$. In particular, we have that the codeword $\hat{c}$ that maximises $\mathbb{P}(c \mid m)$ is precisely the codeword that minimises $d(m,c)$.

---
#### Example.

Let us consider the example $\mathcal{C} = \{000,111\}$ again. Suppose we receive the message $m=100$. We know from previous computations that the most likely codeword that was sent is $c=000$. We can recompute this using the Hamming distance. In particular, we have that
$$ d(m=100,c=000) = 1 \; \text{ and } \; d(m=100,c=111) = 2 $$
and so $c=000$ minimises the Hamming distance.

As before we can compute the Hamming distance and most likely codeword for all possible messages $m \in \{0,1\}^3$.
| Message $m$   | $d(m, c=000)$ | $d(m, c=111)$ | Most likely codeword $c$ |
| ---                    | ---                    | ---                    | ---             |
| 000                    | 0        | 3         | 000  |
| 100, 010, 001          | 1         | 2         | 000  |
| 110, 101, 011          | 2         | 1         | 111  |
| 111                    | 3         | 0        | 111  |
---

<!-- 

Moreover, since the log function is monotonically increasing maximising $\mathbb{P}(m \mid c)$ is equivalent to maximising $\mathrm{log}(\mathbb{P}(m \mid c)) $. (TODO: make this a more natural jump.) By applying the log function we obtain
$$ \mathrm{log}(\mathbb{P}(m \mid c)) = \mathrm{log}(\prod_i \mathbb{P}(m_i \mid c_i)) = \sum_i \mathrm{log}(\mathbb{P}(m_i \mid c_i)). $$
Now recall that for a binary symmetric channel we assume that 
In particular, it depends only on when bits are the same and when they are not, similar to the Hamming distance. Thereofore, by using the Hamming distance between two codewords we obtain
$$ \sum_i \mathrm{log}(\mathbb{P}(m_i \mid c_i)) = d(c,m)\mathrm{log}(p) + (1-d(c,m))\mathrm{log}(1-p). $$
Simplifying this we obtain
$$ \sum_i \mathrm{log}(\mathbb{P}(m_i \mid c_i)) =  d(c,m)\mathrm{log}(\frac{p}{1-p}) + \mathrm{log}(1-p). $$
Notice that $\mathrm{log}(1-p)$ does not depend on $c$, so maximising $\mathbb{P}(c \mid m)$ is equivalent to maximising $$d(c,m)\mathrm{log}(\frac{p}{1-p})$$. Since we assume that $p < 0.5$, we have that $\mathrm{log}(\frac{p}{1-p})$ is negative. Thus, maximising over $d(c,m)\mathrm{log}(\frac{p}{1-p})$ is equivalent to minimsiing over $d(c,m)$. Therefore, we have that
$$ \mathrm{max}_{c \in \mathcal{C}} (\mathbb{P}(c \mid m)) = \mathrm{min}_{c \in \mathcal{C}} (d(c,m)).$$

#### Example

Take the previous example with the code $\mathcal{C} = \{000,111\}$. We can easily compute the Hamming distances between any message $m$ and codeword $c$ to obtain the following table:

| Message $m$   | $d(m, c=000)$ | $d(m, c=111)$ | Closest = most likely codeword |
| ---                    | ---                    | ---                    | ---             |
| 000                    | 0        | 3         | 000  |
| 100, 010, 001          | 1         | 2         | 000  |
| 110, 101, 011          | 2         | 1         | 111  |
| 111                    | 3         | 0        | 111  | 

---
-->
Using the Hamming distance makes it faster to apply the maximum likelihood decoder. However, for a code $\mathcal{C}$ with rank $k$, it still requires computing and comparing $2^k$ different numbers. Therefore, as the code gets bigger the Maximum Likelihood Decoder becomes computationally impractical. We need more efficient algorithms that estimate the most likely codeword without having to compute the Hamming distances explicitly.

In [ ]:
from linearcodes import Codeword, LinearCode, HammingCode
import random
import numpy as np
import time

def send_codeword(
        code_length: int,
        channel_probabilities: list[float],
        codeword: Codeword):
    if len(channel_probabilities) != code_length:
        raise ValueError(f"List of probabilities must be of length code_length={code_length}. Got {len(channel_probabilities)}.")
    if len(codeword) != code_length:
        raise ValueError(f"Codeword must of length code_length={code_length}. Got {len(codeword)}.")
    message = []
    for bit_idx in range(code_length):
        p = channel_probabilities[bit_idx]
        c = codeword.vector[bit_idx]
        r = np.random.uniform(0,1)
        if r < p:
            flip_bit = (c+1)%2
            message.append(flip_bit)
        else:
            message.append(c)
    return message

def hamming_distance(x: list[int], y: list[int]) -> int:
    if len(x) != len(y):
        raise ValueError("Both x and y must be the same length.")
    return sum([1 for bit_idx in range(len(x)) if x[bit_idx] != y[bit_idx]])


def minimise_hamming_distance(code_length: int,
                              codewords: list[Codeword],
                              message: list[int]) -> Codeword:
    """
    Returns the codeword that was most likely to be sent given message is received using Hamming distance.
    Args:
        - channel_probabilities (list[float]):
            entry with index i is the probability bit i will flip.
        - code (lc.LinearCode):
            code used for encoding.
        - message (list[int]):
            received message.
    """
    if len(message) != code_length:
        raise ValueError(f"Message must be of length {code_length}. Got {len(message)}.")
    most_likely_codeword = Codeword([0]*code_length)
    min_hamming_dist = code_length
    for c in codewords:       
        hamming_dist = hamming_distance(c.vector, message)
        if hamming_dist < min_hamming_dist:
            most_likely_codeword = c
            min_hamming_dist = hamming_dist
    return most_likely_codeword

code = HammingCode(3)
code_length = code.length
codewords = code.codewords
channel_probabilities = [float(0.1)]*code_length
results: dict[int, tuple[bool, float]] = {}

num_trials = 10
for trial in range(num_trials):
    print(f"\n=== Minimise Hamming distance decoder {trial+1} ===")
    codeword = random.choice(codewords)

    message = send_codeword(code_length, channel_probabilities, codeword)
    start_time = time.time()
    most_likely_codeword = minimise_hamming_distance(code_length, codewords, message)
    decode_time = time.time() - start_time

    print(f"\nRows are: sent codeword c, received message m, most likely sent codeword c'.")
    data = np.array([codeword.vector, message, most_likely_codeword.vector])
    print(data)

    decode_correct = codeword == most_likely_codeword
    print(f"\nDecoding correct? {decode_correct}")
    print(f"Actual number of errors: {sum(1 for bit_idx in range(code_length) if codeword.vector[bit_idx] != message[bit_idx])}")
    print(f"Guessed number of errors: {sum(1 for bit_idx in range(code_length) if most_likely_codeword.vector[bit_idx] != message[bit_idx])}")

    results[trial] = (decode_correct, decode_time)
    print(f"\nDecode correct? {decode_correct}")
    print(f"Time to decode = {decode_time*(10**5):.3f}x10^5")

#### Decoding with parity checks

Recall a linear code $\mathcal{C}$ has a parity check matrix $P$ such that for any vector $v$ in the ambient space of $\mathcal{C}$ we have that $Pv = 0$ if and only if $v$ is a codeword. We can use the parity check matrix to simplify the maximal likelihood method above.

Recall in an ideal world for a message $m$ we want to compute $\mathbb{P}(c \mid m)$ for all codewords $c \in \mathcal{C}$ to find the most likely intended message and decode $m$. Currrently this requires $2^k$ computations where $k$ is the rank of the code. However, we can reduce this drastically by instead considering whether each of the individual bits are correct. 

First let us assume that no column in the parity check matrix $P$ is all 0's. If this is the case then puncturing at this index results in a code $\mathcal{C}'$ that has the same rank as $\mathcal{C}$ but length one smaller ... TODO: prove this somewhere. Now we claim that 

$$  = \mathbb{P}(c \mid m) = \mathbb{P}(c \mid m \text{ and } Pc = 0) = \mathbb{P}(\bigcap_{i=1}^n c_i \mid m \text{ and } Pc = 0) $$

$$ \mathbb{P}(c \text{ and } m \text{ and } Pc = 0) = \mathbb{P}(c \text{ and } \bigcap_{i=1}^n m_i \text{ and } Pc = 0) $$
$$ = \mathbb{P}(m_1 \mid c \text{ and } \bigcap_{i=2}^n m_i \text{ and } Pc = 0 )\mathbb{P}(c \text{ and } \bigcap_{i=2}^n m_i \text{ and } Pc = 0 ) $$

$$ \mathbb{P}(\bigcap_{i=1}^n c_i \text{ and } m \text{ and } Pc = 0) = \mathbb{P}(c_1 \mid \bigcap_{i=2}^n c_i \text{ and } m \text{ and } Pc = 0)\mathbb{P}(\bigcap_{i=2}^n c_i \text{ and } m \text{ and } Pc = 0) $$

$$= (\prod_{j=1}^{n-1} \mathbb{P}(c_j \mid \bigcap_{i=j+1}^n c_i \text{ and } m \text{ and } Pc = 0)) \cdot \mathbb{P}(c_n \text{ and } m \text{ and } Pc = 0)$$

$$ \mathbb{P}(c_i \mid m) = \frac{}

#### Distance of a code

We have already seen that we cannot with 100% certainty determine the correct codeword for a message $m$. However, if we know the number of errors that occured during transmission sometimes we can. The _distance_ of a code is 
A priori it is not clear that there is a unique most likely codeword for a message $m$. In fact, in general there is not.



#### Repetition Codes

In [ ]:
from linearcodes import RepetitionCode
import numpy as np
import random

# C1 = {000,111}

C1 = RepetitionCode(3)
num_trials = 4
probs = np.random.uniform(0.2,0.6,3).tolist()
print(f"Probabilities of bit-flip: {probs}")
message_space = C1.message_space()
for n in range(num_trials):
    message = random.choice(message_space)
    encoded, received, decoded = C1.send_and_decode_message(message, probs, verbose=True)
    print(f"\n=== New transmission ===")
    print(f"Message to send: {message}.")
    print(f"Encoded message: {encoded}.")
    print(f"Received message: {received}.")
    print(f"Decoded message: {decoded}.")
    print(f"Message decoded successfully: {message == decoded}.")


In [ ]:
codelength = 21
C1 = RepetitionCode(codelength)
num_trials = 4
probs = np.random.uniform(0.2,0.5,codelength).tolist()
print(f"Probabilities of bit-flip: {probs}")
message_space = C1.message_space()
for n in range(num_trials):
    message = random.choice(message_space)
    encoded, received, decoded = C1.send_and_decode_message(message, probs, verbose=True)
    print(f"\n=== New transmission ===")
    print(f"Message to send: {message}.")
    print(f"Encoded message: {encoded}.")
    print(f"Received message: {received}.")
    print(f"Decoded message: {decoded}.")
    print(f"Message decoded successfully: {message == decoded}.")

#### Hamming Codes

#### LDPC Codes

An LDPC code is a linear code whose parity check matrix is sparse, meaning each parity check equation involves only a few variables. This sparsity makes it efficient to check whether a codeword contains an error. The parity checks can be visualised using a Tanner graph, a bipartite graph in which one set of nodes represents the bits in a codeword and the other set represents the parity check equations. A bit node is connected to a check node if that bit appears in the corresponding parity check equation.

To define an LDPC code, one specifies the code length (number of bits), the number of parity check equations, and the weight of each parity check (the number of bits involved in the equation).

## Belief propagation

Belief propagation is a decoder that estimates the likelihood of an error by using the Tanner graph of a code. At each stage the algorithm chooses a bit vertex and a parity check vertex that are connected and asks:
    What is the probability that the sent bit is a 0 or a 1 given the other received bits in the parity check?

This algorithm involves running over different bits and parity checks updating the probabilities the sent bit was $0$ or $1$. The algorithm is suited to Tanner graphs that are sparse, since then only a few bits influence each parity check equation. Therefore, this is a decoder often used for LDPC codes. The main difficulty is deciding which bit and parity check to use at each step. If the same nodes are used too often then the influence of the rest of the graph is lost and the estimates become less reliable.

Assume the code has length $n$ and we receive a message $r_1 r_2 \dots r_n$. Let us denote the message that was sent by $s_1 s_2 \dots s_n$. We first calculate the probability that each of the received bits is correct based on the probabilities that there will be a bit-flip in the channel. Assume the channel is symmetric, so the bit-flip probability is the same whether the bit is $0$ or $1$. Let the flip probability be $p$. The bit $r$ is correct if and only if no flip occurs, so  
$$
\mathbb{P}(r \text{ is correct}) = 1 - p \; \text{ and } \; \mathbb{P}(r \text{ is incorrect}) = p.
$$
Now a single step of the belief propagation algorithm goes as follows:
1. Choose one received bit $r_i$.  
2. Choose a parity check $P$ that involves $r_i$, that is  
   $$
   P=\sum_{j\in J} r_j
   $$  
   for some set $J$ with $i\in J$.  
3. Update the probability that $s_i$ is a $0$ or a $1$ in light of the requirement given that the parity check $P$ needs to be satisified. That is, we want to compute $$\mathbb{P}(s_i = 0 \mid P \text{ is satisfied.})  \; \text{ and } \; \mathbb{P}(s_i = 1\mid P \text{ is satisfied.}).$$ The key observation is that $P$ is satisfied if and only if  $$r_1 = \sum_{j \neq 1, j\in J} r_j.$$  This relation, together with the current probabilities for the bits $r_2,\dots,r_n$, determines the updated probability for $r$.

$$ \mathbb{P}(s_1 = r_1 \mid P) = \frac{\mathbb{P}(s_1 = r_1 \text{ and } P)}{\mathbb{P}(P)} $$
If $|J| = 2$.
$$ \mathbb{P}(s_1 = r_1 \mid s_1 = s_2 ) = \frac{\mathbb{P}(s_1 = r_1 \text{ and } s_1 = s_2 )}{\mathbb{P}(s_1 = s_2)} $$


$C = \{110, 111, 001, 000\}$ the probability any bit is $0$ or $1$ is $50-50$; ollie's clever rank nullity argument.

so $P = x_1 + x_2$, say $p_1,p_2,p_3 = 0.4, 0.8, 0.3$.
Suppose we receive $r = 101$. 
What is the probability the $s_2$ is a $0$ and a $1$ given that we know:
1. The parity check must be satisfied i.e. $s_1 + s_2 = 0$.
2. The probability that $s_1 = r_1$ is $0.5$ and $s_3 = r_3$ is $0.7$ i.e. no flips.

So what is the proababilities:
$P(s_2 = 0| s_1 + s_2 = 0, r_1, r_2)$ $P(s_2 = 1| s_1 + s_2 = 0, r_1, r_2)$

$$ \mathbb{P}(s_2 = 0| s_1 + s_2 = 0, r_1, r_2) = \frac{\mathbb{P}(s_2=0 \text{ and }  s_1 + s_2 = 0, r_1, r_2)}{\mathbb{P}(s_1 + s_2 = 0, r_1, r_2)} $$



$P(s_1 = s_2)$
If $r_1 = r_2$ it is the probability neither $r_1$ nor $r_2$ were flipped or both $r_1$ and $r_2$ were flipped.
$P(r_1 \text{ flip})P(r_2 \text{ flip}) = $ 